In [25]:
import string
from collections import defaultdict
import json

import requests
from bs4 import BeautifulSoup
from requests.compat import urljoin
from tqdm import tqdm

In [2]:
root = "https://www.payscale.com"

# Scrape the list of skill categories

In [3]:
skill_main = "research/US/Skill"
url = urljoin(root, skill_main)
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
skill_cats = [e["href"] for e in soup.find_all(attrs={"class": "related-content-card"})]

# Scrape the skills for each category
Skills are listed by alphabets and some has multiple pages

In [22]:
skills = set()
a_to_z = string.ascii_uppercase + "#"

skills = defaultdict(set)
for skill_cat in tqdm(skill_cats):
    skill_cat_txt = skill_cat.split("/")[-1]
    for char in a_to_z:
        for n in range(1, 4):
            url = f"{root}{skill_cat}/{char}/Page-{n}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            for a in soup.find_all(attrs={"class": "subcats__links__item"}):
                skills[skill_cat_txt].add(a.text)

100%|██████████| 27/27 [16:19<00:00, 36.28s/it]


# Save the skills into JSON

In [29]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError


with open("skills.json", "w") as f:
    json.dump(skills, f, default=set_default, indent=4)